# Partial Variable


## 原始用法

In [17]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

msgs = [
    ("system", "你是一个有氧拳击教练，请帮我解释拳击术语，如果术语是英文，请先翻译为中文。"
               "{format_instructions}"),
    ("human", "术语：{name}")
]

format_instructions = """结果请按以下形式输出为 json 格式:

{
	"terminology": string  // 中文术语名称
	"explanation": string  // 术语的解释
}
"""

prompt = ChatPromptTemplate.from_messages(msgs).partial(format_instructions=format_instructions)

chat_model = ChatOpenAI(model_name="openai/gpt-4o-mini")
chain = prompt | chat_model

resp = chain.invoke({"name": "下潜躲避"})
print(resp.content)


{
	"terminology": "下潜躲避",
	"explanation": "下潜躲避是一种拳击技术，指的是拳击手通过弯曲膝盖和下沉身体来躲避对手的攻击。这个动作可以帮助拳击手降低重心，从而避免被击中，同时也为反击创造机会。"
}


## 配合 OutputParser 规范输出

In [22]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field


class Work(BaseModel):
    title: str = Field(description="作品名称")
    description: str = Field(description="作品简介")

parser = JsonOutputParser(pydantic_object=Work)

prompt = PromptTemplate(
    template="请列举 3 部 {author} 的作品。\n{format_instructions}",
    input_variables=["author"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chat = ChatOpenAI(model_name="openai/gpt-4o-mini")
chain = prompt | chat | parser

resp = chain.invoke({"author": "鲁迅"})
print(resp, "\n\n", type(resp), "\n", len(resp), "\n", type(resp[0]))


[{'title': '狂人日记', 'description': '这是鲁迅的第一篇白话文小说，通过一个精神病患者的视角，揭示了社会的黑暗与人性的扭曲。'}, {'title': '阿Q正传', 'description': '这部作品通过阿Q这一角色，讽刺了中国社会的劣根性，深刻反映了那个时代人们的精神状态。'}, {'title': '呐喊', 'description': '《呐喊》是鲁迅的短篇小说集，集中展现了社会的压迫和个人的无奈，是中国现代文学的重要作品。'}] 

 <class 'list'> 
 3 
 <class 'dict'>


# Few Shot Prompt Template

In [20]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model_name="openai/gpt-4o-mini")

samples = [
    {
        "flower_type": "玫瑰",
        "occasion": "爱情",
        "ad_copy": "玫瑰，浪漫的象征，是你向心爱的人表达爱意的最佳选择。"
    },
    {
        "flower_type": "康乃馨",
        "occasion": "母亲节",
        "ad_copy": "康乃馨代表着母爱的纯洁与伟大，是母亲节赠送给母亲的完美礼物。"
    },
    {
        "flower_type": "百合",
        "occasion": "庆祝",
        "ad_copy": "百合象征着纯洁与高雅，是你庆祝特殊时刻的理想选择。"
    },
    {
        "flower_type": "向日葵",
        "occasion": "鼓励",
        "ad_copy": "向日葵象征着坚韧和乐观，是你鼓励亲朋好友的最好方式。"
    }
]

template = "鲜花类型: {flower_type}\n场合: {occasion}\n文案: {ad_copy}"

prompt = FewShotPromptTemplate(
    example_prompt=PromptTemplate.from_template(template),
    examples=samples,
    suffix="鲜花类型: {flower_type}\n场合: {occasion}",
)
res = prompt.invoke({"flower_type": "野玫瑰", "occasion": "爱情"})
print(res.to_string(), "\n\n\n")

print(chat.invoke(res).content)


鲜花类型: 玫瑰
场合: 爱情
文案: 玫瑰，浪漫的象征，是你向心爱的人表达爱意的最佳选择。

鲜花类型: 康乃馨
场合: 母亲节
文案: 康乃馨代表着母爱的纯洁与伟大，是母亲节赠送给母亲的完美礼物。

鲜花类型: 百合
场合: 庆祝
文案: 百合象征着纯洁与高雅，是你庆祝特殊时刻的理想选择。

鲜花类型: 向日葵
场合: 鼓励
文案: 向日葵象征着坚韧和乐观，是你鼓励亲朋好友的最好方式。

鲜花类型: 野玫瑰
场合: 爱情 



文案: 野玫瑰，散发着自然的魅力与独特的个性，是向那个特别的人表达真挚爱意的别致选择。


## 使用示例选择器

In [17]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain_qdrant import QdrantVectorStore
from langchain_openai import ChatOpenAI
from langchain_ollama import OllamaEmbeddings

chat = ChatOpenAI(model_name="openai/gpt-4o-mini")
embedding = OllamaEmbeddings(model="nomic-embed-text", base_url="http://localhost:11434")

samples = [
    {
        "flower_type": "玫瑰",
        "occasion": "爱情",
        "ad_copy": "玫瑰，浪漫的象征，是你向心爱的人表达爱意的最佳选择。"
    },
    {
        "flower_type": "康乃馨",
        "occasion": "母亲节",
        "ad_copy": "康乃馨代表着母爱的纯洁与伟大，是母亲节赠送给母亲的完美礼物。"
    },
    {
        "flower_type": "百合",
        "occasion": "庆祝",
        "ad_copy": "百合象征着纯洁与高雅，是你庆祝特殊时刻的理想选择。"
    },
    {
        "flower_type": "向日葵",
        "occasion": "鼓励",
        "ad_copy": "向日葵象征着坚韧和乐观，是你鼓励亲朋好友的最好方式。"
    }
]

template = "鲜花类型: {flower_type}\n场合: {occasion}\n文案: {ad_copy}"
example_selector = SemanticSimilarityExampleSelector.from_examples(samples, embedding, QdrantVectorStore, k=1)

prompt = FewShotPromptTemplate(
    example_prompt=PromptTemplate.from_template(template),
    example_selector=example_selector,
    suffix="鲜花类型: {flower_type}\n场合: {occasion}",
)
res = prompt.invoke({"flower_type": "野玫瑰", "occasion": "爱情"})

print(res.to_string(), "\n\n\n")

print(chat.invoke(res).content)

鲜花类型: 玫瑰
场合: 爱情
文案: 玫瑰，浪漫的象征，是你向心爱的人表达爱意的最佳选择。

鲜花类型: 野玫瑰
场合: 爱情 



文案: 野玫瑰，虽不如红玫瑰般夺目，却散发着自然的纯真与坚韧，寓意着真实而不拘一格的爱情。送上一束野玫瑰，让心爱的人感受到你对她独特而深沉的爱意。


In [19]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

example_selector = SemanticSimilarityExampleSelector.from_examples(samples, embedding, QdrantVectorStore, k=1)

prompt = FewShotPromptTemplate(
    example_prompt=PromptTemplate.from_template(template),
    example_selector=example_selector,
    suffix="鲜花类型: {flower_type}\n场合: {occasion}",
)
res = prompt.invoke({"flower_type": "野玫瑰", "occasion": "爱情"})

print(res.to_string(), "\n\n\n")

print(chat.invoke(res).content)

鲜花类型: 康乃馨
场合: 母亲节
文案: 康乃馨代表着母爱的纯洁与伟大，是母亲节赠送给母亲的完美礼物。

鲜花类型: 野玫瑰
场合: 爱情 



文案: 野玫瑰象征着热烈与自由，传达出真挚的爱意，是表达爱情的最佳选择。在这个特别的时刻，让野玫瑰为你的心声添上浪漫的色彩。
